## הגדרות ומקדים

In [21]:


import pandas as pd
import geopandas as gpd
import numpy as np
from shapely import wkt
from matplotlib import pyplot as plt 
import folium
import fiona
from shapely.geometry import Point


In [22]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.3f' % x)


## פונקציות

In [23]:
def double_taz_num(df):
    dup_taz_num=df.groupby(['Taz_num']).size().reset_index(name='count').query('count>1').Taz_num.to_list()
    return df.loc[df['Taz_num'].isin(dup_taz_num)]

In [24]:
def make_point(df):
    df_point=df.copy()
    df_point['centroid'] = df_point.representative_point()
    df_point=df_point.set_geometry('centroid')
    df_point=df_point.drop(columns=['geometry'],axis=1)
    return df_point

In [25]:
def up_load_gdb(path,layer_name):
    path='{}'.format(path)
    layer_list=fiona.listlayers(path)
    gpd_layer=gpd.read_file(path, layer=layer_list.index(layer_name))
    return gpd_layer

In [26]:
def up_load_shp(path):
    path='{}'.format(path)
    gpd_layer=gpd.read_file(path)
    return gpd_layer

In [27]:
def unique_id_in_one_taz(df_to_geoode,unique_field,gpd_for_geocode,taz):
    
    code_to_find=list(df_to_geoode[unique_field].unique())

    gpd_for_geocode_to_sum_by_taz=gpd_for_geocode[[unique_field,'geometry']].loc[gpd_for_geocode[unique_field].isin(code_to_find)]

    gpd_for_geocode_to_sum_by_taz=gpd.sjoin(taz[['Taz_num','geometry']],gpd_for_geocode_to_sum_by_taz)

    gpd_for_geocode_in_one_taz=list(gpd_for_geocode_to_sum_by_taz.groupby(unique_field)[['Taz_num']].nunique().query('Taz_num==1').reset_index()[unique_field])

    return gpd_for_geocode_in_one_taz

In [28]:
def up_load_df(folder_path,file_name):
    
    path_df=r'{}\{}.xlsx'.format(folder_path,file_name)
    df=pd.read_excel(path_df)
    df=df.dropna(how='all')

    return df


In [29]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]


## להריץ תלמידים

In [30]:
%run "students_current_year_230706.ipynb"

TypeError: 'GeometryArray' with dtype geometry does not support reduction 'sum'

TypeError: 'GeometryArray' with dtype geometry does not support reduction 'sum'

## להעלות שכבות

In [ ]:
#העלת משתנים להרצת הקוד
df_inputs_outputs = pd.read_excel('inputs_outputs.xlsx')

software_data_folder_location=df_inputs_outputs['location'][0]

forecast_version_folder_location=df_inputs_outputs['location'][1]


In [ ]:
path=r'{}\background_files\EMP_KIBOLET.gdb'.format(software_data_folder_location)
EMP_kibolet=up_load_gdb(path,'EMP_kibolet')

In [ ]:
EMP_kibolet=EMP_kibolet.fillna(0)

In [ ]:
EMP_kibolet['emp_without_palestin']=EMP_kibolet['kayim_emp']-EMP_kibolet['Palestinians']

In [ ]:
emp_without_palestin_not_okev=EMP_kibolet['emp_without_palestin'].sum().item()

In [ ]:
emp_Education=taz.query('main_secto!="Palestinian"')['emp_Education'].sum().item()

## הוספת סטודנטים לאוכלוסיה ולהתפלגות גילים

In [ ]:
student_dorms=up_load_shp(r'{}\background_files\student_dorms.shp'.format(software_data_folder_location))

In [ ]:
taz['student_dorms']=gpd.sjoin(taz[['geometry']].reset_index(),student_dorms).pivot_table(index='Taz_num',aggfunc=sum)[['student_nu']]

In [ ]:
taz=taz.fillna(0)

In [ ]:
taz['pop_without_dorms_yeshiva']=taz['pop']

In [ ]:
taz['pop']=taz['pop']+taz['student_dorms']

In [ ]:
taz['pop_20_just_from_aprt']=taz['pop_20']

In [ ]:
taz['pop_25_just_from_aprt']=taz['pop_25']

In [ ]:
taz['pop_20']=taz['pop_20']+taz['student_dorms']*0.6

In [ ]:
taz['pop_25']=taz['pop_25']+taz['student_dorms']*0.4

In [ ]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## הוספת תלמידי ישיבה (באזורים חרדיים) אל כמות אוכלוסיה,התפלגות גילים 

In [ ]:
taz['pop_15_just_from_aprt']=taz['pop_15']

In [ ]:

taz.loc[taz['main_secto']=="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']=="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

In [ ]:
col=['pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

taz.loc[taz['main_secto']!="Palestinian"].loc[taz['pop_check']!=0]

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_15_just_from_aprt
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## כימות מועסקים במרחב צתאל

In [ ]:
pre_woman=0.5

In [ ]:
pre_man=1-pre_woman

In [ ]:
work_age=[ 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_50',
 'pop_55',
 'pop_60']

In [ ]:
under_work_age=[ 'pop_15', 'pop_20']

In [ ]:
over_work_age=[ 'pop_65', 'pop_70', 'pop_75up']

In [ ]:
taz['work_age']=taz[work_age].sum(axis=1)

taz['under_work_age']=taz[under_work_age].sum(axis=1)

taz['over_work_age']=taz[over_work_age].sum(axis=1)

In [ ]:
sector='U_Orthodox'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.75+taz['work_age']*pre_man*0.55
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.07+taz['under_work_age']*pre_man*0.09
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='Jewish'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.9+taz['work_age']*pre_man*0.92
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.20+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.2+taz['over_work_age']*pre_man*0.15

sector='Arab'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09

sector='arabs_behined_seperation_wall'

taz.loc[taz['main_secto']==sector,'pop_emp']=taz['work_age']*pre_woman*0.25+taz['work_age']*pre_man*0.7
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['under_work_age']*pre_woman*0.2+taz['under_work_age']*pre_man*0.15
taz.loc[taz['main_secto']==sector,'pop_emp']=taz['pop_emp']+taz['over_work_age']*pre_woman*0.05+taz['over_work_age']*pre_man*0.09


## חישוב אבטלה

In [ ]:
taz['pop_emp_employed']=0

In [ ]:
taz.loc[taz['main_secto']=="U_Orthodox",'pop_emp_employed']=taz['pop_emp']*0.97

In [ ]:
taz.loc[taz['main_secto']=="Jewish",'pop_emp_employed']=taz['pop_emp']*0.98

In [ ]:
taz.loc[taz['jew']==0,'pop_emp_employed']=taz['pop_emp']*0.95

## חישוב יוממות

In [ ]:
commuting=up_load_shp(r'{}\background_files\commuting_230712.shp'.format(software_data_folder_location))

In [ ]:
taz['commuting']=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],commuting[['commuting', 'geometry']]).set_index('Taz_num')['commuting']/100

In [ ]:
taz['pop_emp_employed_out_of_jtmt_area']=taz['pop_emp_employed']*taz['commuting']

In [ ]:
emp_left_jtmt_area=taz['pop_emp_employed_out_of_jtmt_area'].sum().item()

In [ ]:
round(emp_left_jtmt_area,-3)

94000.0

In [ ]:
emp_from_jtmt_area=taz['pop_emp_employed'].sum().item()-emp_left_jtmt_area

In [ ]:
emp_in_jtmt_area=emp_from_jtmt_area*1.07 #יוממות פנימה מחוץ למרחב

In [ ]:
round(emp_in_jtmt_area,-3)

510000.0

In [ ]:
emp_in_jtmt_area_without_mobile=emp_in_jtmt_area*0.95 #הפחחת עובדים ניידים

In [ ]:
round(emp_in_jtmt_area_without_mobile,-3)

485000.0

In [ ]:
emp_okev=emp_in_jtmt_area_without_mobile-emp_Education-emp_without_palestin_not_okev

## פיזור עוקב

In [ ]:
EMP_kibolet['geometry_buff']=EMP_kibolet.buffer(250)

In [ ]:
EMP_buffer=EMP_kibolet.set_geometry('geometry_buff').query('kayim_emp>0')

In [ ]:
EMP_buffer=EMP_buffer.dissolve()[['geometry_buff']]

In [ ]:
taz['taz_area']=taz.area

In [ ]:
emp_buffer_taz=gpd.overlay(EMP_buffer,taz.reset_index())

In [ ]:
emp_buffer_taz['emp_samll_area']=emp_buffer_taz.area

In [ ]:
emp_buffer_taz['emp_pre_from_taz']=emp_buffer_taz['emp_samll_area']/emp_buffer_taz['taz_area']

In [ ]:
taz_num_no_need_okev=emp_buffer_taz.loc[emp_buffer_taz['emp_pre_from_taz']>0.4].Taz_num.to_list()

In [ ]:
taz['okev']=0
taz.loc[(~taz.index.isin(taz_num_no_need_okev))&(taz['pop']>0)&(taz['main_secto']!="Palestinia"),'okev']=1

In [ ]:
taz.loc[taz['main_secto']=="arabs_behined_seperation_wall",'okev']=1

In [ ]:
city_muni=['מודיעין עילית','בית שמש','ירושלים','מודיעין - מכבים - רעות']

## מקדם עוקב

In [ ]:
okev_level=0.15

In [ ]:
arab_city=taz.query('jew==0 & okev==1')['aprt_20'].sum().item()

In [ ]:
arab_sub=0

In [ ]:
ou_sub=taz.query('main_secto=="U_Orthodox" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [ ]:
ou_city=taz.query('main_secto=="U_Orthodox" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [ ]:
general_city=taz.query('main_secto=="Jewish" & Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [ ]:
general_sub=taz.query('main_secto=="Jewish" & ~Muni_Heb.isin(@city_muni) & okev==1')['aprt_20'].sum().item()

In [ ]:
base_okev_factor=(emp_okev-(okev_level*general_city-okev_level*(ou_sub+arab_city)-2*okev_level*arab_sub))/(general_sub+general_city+ou_city+ou_sub+arab_city+arab_sub)

In [ ]:
base_okev_factor=round(base_okev_factor,3)

In [ ]:
data = [['Jewish', base_okev_factor,base_okev_factor+okev_level],['arab', base_okev_factor-okev_level,base_okev_factor], ['U_Orthodox',base_okev_factor-2*okev_level,base_okev_factor-okev_level]]
 
# Create the pandas DataFrame
df = pd.DataFrame(data, columns=['sector', 'sub','city'])
okev_factor=df.set_index('sector')
okev_factor.to_excel(r'{}\Intermediates\okev_factor.xlsx'.format(software_data_folder_location))

In [ ]:
taz['emp_okev']=0

In [ ]:
taz.loc[(taz['jew']==0)&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['arab','city']

In [ ]:
taz.loc[(taz['main_secto']=="U_Orthodox")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','sub']

In [ ]:
taz.loc[(taz['main_secto']=="U_Orthodox")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['U_Orthodox','city']

In [ ]:
taz.loc[(taz['main_secto']=="Jewish")&(taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','city']

In [ ]:
taz.loc[(taz['main_secto']=="Jewish")&(~taz['Muni_Heb'].isin(city_muni))&(taz['okev']==1),'emp_okev']=taz['aprt_20']*okev_factor.loc['Jewish','sub']

In [ ]:
taz.emp_okev.sum().item()-emp_okev

-1631.8376567774103

## פיצול שכבת לא עוקב לאזורי תנועה

In [ ]:
EMP_kibolet['emp_area']=EMP_kibolet.area

In [ ]:
EMP_kibolet['ID']=EMP_kibolet.index

In [ ]:
EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)

C:\Users\gidon\AppData\Local\Temp\ipykernel_27552\35171637.py:1: UserWarning: `keep_geom_type=True` in overlay resulted in 280 dropped geometries of different geometry types than df1 has. Set `keep_geom_type=False` to retain all geometries
  EMP_kibolet_by_taz=gpd.overlay(taz.reset_index()[['Taz_num','taz_area','geometry']],EMP_kibolet)


In [ ]:
EMP_kibolet_by_taz['small_area']=EMP_kibolet_by_taz.area

In [ ]:
EMP_kibolet_by_taz['pre_small_area_emp']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['emp_area']

In [ ]:
EMP_kibolet_by_taz['pre_small_area_taz']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['taz_area']

In [ ]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.query('(pre_small_area_taz >0.7) | (pre_small_area_emp >0.1)').drop(columns='geometry_buff')

In [ ]:
EMP_kibolet_by_taz=EMP_kibolet_by_taz.set_index('ID')

In [ ]:
EMP_kibolet_by_taz['id_area_for_pre']=EMP_kibolet_by_taz.reset_index().groupby(by='ID').sum()['small_area']

In [ ]:
EMP_kibolet_by_taz['prec_from_id']=EMP_kibolet_by_taz['small_area']/EMP_kibolet_by_taz['id_area_for_pre']

In [ ]:
EMP_kibolet_by_taz['emp_without_palestin_in_taz']=EMP_kibolet_by_taz['emp_without_palestin']*EMP_kibolet_by_taz['prec_from_id']

In [ ]:
taz['emp_not_okev']=EMP_kibolet_by_taz.pivot_table(index='Taz_num',aggfunc=sum)[['emp_without_palestin_in_taz']]

In [ ]:
taz=taz.fillna(0)

In [ ]:
taz['total_emp']=taz['emp_not_okev']+taz['emp_okev']+taz['emp_Education']

In [ ]:
taz.query('main_secto!="Palestinian"')['total_emp'].sum()-emp_in_jtmt_area_without_mobile

-1631.8376567774103

## תוצאות

In [ ]:
taz.groupby(by='zonetype').sum()[['total_emp']]#.loc[['Jerusalem','Judea and Samaria']]

,total_emp
zonetype,
Jerusalem,372399.821
Judea and Samaria,82220.279
Palestinian,18.000
Ramla,28263.971


In [ ]:
taz.groupby(by='Muni_Heb').sum()[['total_emp']].loc[['ירושלים','בית שמש','ביתר עילית','מודיעין - מכבים - רעות','מעלה אדומים','מודיעין עילית']]

,total_emp
Muni_Heb,
ירושלים,324049.187
בית שמש,26987.510
ביתר עילית,7146.349
מודיעין - מכבים - רעות,25654.360
מעלה אדומים,7653.404
מודיעין עילית,10724.250


In [ ]:
taz.groupby(by=['Muni_Heb','main_secto']).sum()[['total_emp']].loc[['ירושלים']]

total_emp
Muni_Heb main_secto                              
ירושלים  Arab                           57152.917
         Jewish                        198076.972
         U_Orthodox                     55686.526
         arabs_behined_seperation_wall  13132.772

## התפלגות ענפי תעסוקה

In [ ]:
emp_category_type=up_load_shp(r'{}\background_files\emp_category_type.shp'.format(software_data_folder_location)).fillna(0)

In [ ]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [ ]:
emp_category_type[col]=emp_category_type[col]/100

In [ ]:
emp_category_type=gpd.sjoin(make_point(taz.reset_index())[['Taz_num','centroid']],emp_category_type).set_index('Taz_num')

In [ ]:
col=['agri', 'Indus', 'Com_hotel', 'Business', 'Public']

In [ ]:
taz.join(emp_category_type[col]).query('(emp_not_okev>0 | emp_okev>0 )& agri.isna() & main_secto!="Palestinian" ')

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_15_just_from_aprt,work_age,under_work_age,over_work_age,pop_emp,pop_emp_employed,commuting,pop_emp_employed_out_of_jtmt_area,taz_area,okev,emp_okev,emp_not_okev,total_emp,agri,Indus,Com_hotel,Business,Public
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


In [ ]:
taz=taz.join(emp_category_type[col]).fillna(0)

In [ ]:
for i in col:
    taz['{}'.format(i)]=taz['{}'.format(i)]*(taz['emp_not_okev']+taz['emp_okev'])

In [ ]:
taz['check_emp_sum']=taz[col].sum(axis=1)-(taz['emp_not_okev']+taz['emp_okev'])

In [ ]:
taz.query('check_emp_sum>10 |check_emp_sum<-10')

,CR_PNIM,Join_Count,TARGET_FID,Agg_taz_nu,Taz_name,Arab,arabs_behi,Jewish,Palestinia,U_Orthodox,main_secto,ENG_NAME_n,Muni_Heb,Sug_Muni,jeru_metro,zonetype,in_jerusal,SCHN_NAME,geometry,STAT,aprt_20,pop,pop_0,pop_10,pop_15,pop_20,pop_25,pop_30,pop_35,pop_40,pop_45,pop_5,pop_50,pop_55,pop_60,pop_65,pop_70,pop_75up,pop_hardi,precent_of_stat_data,pre_hardi,hh_size,pop_check,student_toddlers,student_gov,delta_student,student_demand,student_demand_left,student_demand_left_sum_by_muni,student_demand_left_pre,cbs_muni_student_left_by_pre_of_demand_left,jew,student_demand_pre,cbs_muni_student_left_by_pre_of_demand,student_for_Control,student_chardi_not_gov,student_arab_not_gov,student,uni_students,emp_from_uni_student,old_student_yeshiva,yeshiva_dorms_pop_15,yeshiva_dorms_pop_20,yeshiva_dorms_pop_25,yeshiva_dorms_pop_sum,emp_from_Yeshiva_student,student_yeshiva,add_from_old_student_yeshiva,kollim_demand,add_from_kollim_demand,student_yeshiva_with_add_from_old,student_yeshiva_and_kollim,emp_from_student,emp_Education,student_dorms,pop_without_dorms_yeshiva,pop_20_just_from_aprt,pop_25_just_from_aprt,pop_15_just_from_aprt,work_age,under_work_age,over_work_age,pop_emp,pop_emp_employed,commuting,pop_emp_employed_out_of_jtmt_area,taz_area,okev,emp_okev,emp_not_okev,total_emp,agri,Indus,Com_hotel,Business,Public,check_emp_sum
Taz_num,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,


## הוספת סטודנטים אל כמות משקי בית

In [ ]:
taz=taz.fillna(0)

In [ ]:
taz['hh']=taz['aprt_20']+taz['student_dorms']/1.5

## הוספת תלמידי ישיבה (באזורים לא חרדיים) אל כמות אוכלוסיה,התפלגות גילים ומשקי בית (לכלל תלמידי הישיבה)

In [ ]:
taz['hh']=taz['hh']+taz['yeshiva_dorms_pop_sum']

In [ ]:
taz.loc[taz['main_secto']!="U_Orthodox",'pop']=taz['pop']+taz['yeshiva_dorms_pop_sum']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_15']=taz['pop_15']+taz['yeshiva_dorms_pop_15']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_20']=taz['pop_20']+taz['yeshiva_dorms_pop_20']

taz.loc[taz['main_secto']!="U_Orthodox",'pop_25']=taz['pop_25']+taz['yeshiva_dorms_pop_25']

## הוספת אוכלוסיה פלסטינאית

In [ ]:
path=r'{}\background_files'.format(software_data_folder_location)
taz_demo_pls_2020=up_load_df(path,'taz_demo_pls_2020_and_pre_growth_till_2050')[['Taz_num','pop_2020']].set_index('Taz_num')

In [ ]:
col=['pop',
'pop_0',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_5',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'emp_Education',
'pop_emp_employed',
'total_emp',
'agri',
'Indus',
'Com_hotel',
'Business',
'Public',
'hh']

In [ ]:
taz.loc[taz['main_secto']=='Palestinian',col]=0

In [ ]:
taz.loc[taz['main_secto']=='Palestinian','pop']=taz['pop']+taz_demo_pls_2020['pop_2020']

## ייצוא לבקרה

In [ ]:
taz=taz.reset_index()

In [ ]:
pre_yeshiva_from_20_30_age=taz[['student_yeshiva_with_add_from_old']].sum().sum()/taz.query('jew==1')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [ ]:
pre_yeshiva_from_20_30_age

0.5261022022178201

In [ ]:
taz.main_secto.unique()

array(['U_Orthodox', 'arabs_behined_seperation_wall', 'Arab', 'Jewish',
       'Palestinian'], dtype=object)

In [ ]:
pre_uni_from_20_30_age=taz[['uni_students']].sum().sum()/taz.query('main_secto!="Palestinian"')[['pop_20_just_from_aprt','pop_25_just_from_aprt']].sum().sum()

In [ ]:
pre_uni_from_20_30_age

0.3034760359349936

In [ ]:
list(taz)

['Taz_num',
 'CR_PNIM',
 'Join_Count',
 'TARGET_FID',
 'Agg_taz_nu',
 'Taz_name',
 'Arab',
 'arabs_behi',
 'Jewish',
 'Palestinia',
 'U_Orthodox',
 'main_secto',
 'ENG_NAME_n',
 'Muni_Heb',
 'Sug_Muni',
 'jeru_metro',
 'zonetype',
 'in_jerusal',
 'SCHN_NAME',
 'geometry',
 'STAT',
 'aprt_20',
 'pop',
 'pop_0',
 'pop_10',
 'pop_15',
 'pop_20',
 'pop_25',
 'pop_30',
 'pop_35',
 'pop_40',
 'pop_45',
 'pop_5',
 'pop_50',
 'pop_55',
 'pop_60',
 'pop_65',
 'pop_70',
 'pop_75up',
 'pop_hardi',
 'precent_of_stat_data',
 'pre_hardi',
 'hh_size',
 'pop_check',
 'student_toddlers',
 'student_gov',
 'delta_student',
 'student_demand',
 'student_demand_left',
 'student_demand_left_sum_by_muni',
 'student_demand_left_pre',
 'cbs_muni_student_left_by_pre_of_demand_left',
 'jew',
 'student_demand_pre',
 'cbs_muni_student_left_by_pre_of_demand',
 'student_for_Control',
 'student_chardi_not_gov',
 'student_arab_not_gov',
 'student',
 'uni_students',
 'emp_from_uni_student',
 'old_student_yeshiva',
 'yes

In [ ]:
col=['Taz_num','Agg_taz_nu','Taz_name','Muni_Heb','jeru_metro',
 'zonetype',
 'in_jerusal','main_secto','aprt_20','student_dorms','student_yeshiva_with_add_from_old','hh','pop_without_dorms_yeshiva',
 'pop', 'hh_size','pop_emp_employed',
 'commuting','student_toddlers',
 'student_gov','cbs_muni_student_left_by_pre_of_demand_left','cbs_muni_student_left_by_pre_of_demand','student_chardi_not_gov',
 'student_arab_not_gov','student','uni_students',
 'student_yeshiva','yeshiva_dorms_pop_15',
 'yeshiva_dorms_pop_20',
 'yeshiva_dorms_pop_25',
 'yeshiva_dorms_pop_sum',
 'add_from_old_student_yeshiva',
 'add_from_kollim_demand', 'emp_from_uni_student','emp_from_Yeshiva_student','emp_from_student',
 'emp_Education','okev',
 'emp_okev',
 'emp_not_okev','total_emp',
 'agri',
 'Indus',
 'Com_hotel',
 'Business',
 'Public']

In [ ]:
file_date=pd.Timestamp.today().strftime('%y%m%d')

In [ ]:
taz[col].to_excel(r'{}\2020_jtmt_forcast_full_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## ייצוא בפורמט של מודלים

In [ ]:
taz['yosh']=0

In [ ]:
taz.loc[taz['zonetype']=='Judea and Samaria','yosh']=1

In [ ]:
taz['jerusalem_city']=0

In [ ]:
taz.loc[taz['in_jerusal']=='yes','jerusalem_city']=1

In [ ]:
taz=taz.reset_index()

In [ ]:
col=['Taz_num',
'yosh',
'jeru_metro',
'jerusalem_city',
'main_secto',
'hh',
'pop',
'pop_0',
'pop_5',
'pop_10',
'pop_15',
'pop_20',
'pop_25',
'pop_30',
'pop_35',
'pop_40',
'pop_45',
'pop_50',
'pop_55',
'pop_60',
'pop_65',
'pop_70',
'pop_75up',
'total_emp',
'Indus',
'Com_hotel',
'Business',
'Public',
'emp_Education',
'agri',
'student',
'uni_students',
'student_yeshiva_and_kollim',
'pop_emp_employed']

In [ ]:
col_new_name=['TAZ',
'yosh',
'in_jerusalem_metropolin',
'jerusalem_city',
'sector',
'hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [ ]:
taz=taz[col]

In [ ]:
taz.columns=col_new_name

In [ ]:
taz.to_excel(r'{}\forecast_2020_{}.xlsx'.format(forecast_version_folder_location,file_date),index=False)

## בקרות לוגיות

In [ ]:
taz.loc[taz['TAZ']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [ ]:
taz.yosh.unique()

array([0, 1], dtype=int64)

In [ ]:
taz.in_jerusalem_metropolin.unique()

array([1., 0.])

In [ ]:
taz.jerusalem_city.unique()

array([1, 0], dtype=int64)

In [ ]:
taz.sector.unique()

array(['U_Orthodox', 'arabs_behined_seperation_wall', 'Arab', 'Jewish',
       'Palestinian'], dtype=object)

In [ ]:
col=['hh_total',
'pop',
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',
'emp_tot',
'indus',
'com_hotel',
'business',
'public',
'education',
'agri',
'student',
'univ',
'UO_Hi_Ed',
'pop_emp_employed']

In [ ]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]<0]['TAZ'])

In [ ]:
taz_to_check

[]

In [ ]:
taz.loc[~(taz['hh_total']<=taz['pop'])]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['hh_total']>0].loc[taz['pop']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed


In [ ]:
col=['age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up',]

taz['pop_check']=round(taz[col].sum(axis=1)-taz['pop'])

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0].query('TAZ==511')#.to_excel('delet.xlsx')#[col].sum(axis=1)

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check


In [ ]:
len(taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_check']!=0])

0

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop']>0].loc[taz['hh_total']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check


In [ ]:
col=[
'age0_4',
'age5_9',
'age10_14',
'age15_19',
'age20_24',
'age25_29',
'age30_34',
'age35_39',
'age40_44',
'age45_49',
'age50_54',
'age55_59',
'age60_64',
'age65_69',
'age70_74',
'age75up']

In [ ]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['pop']<=0]['TAZ'])
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['hh_total']<=0]['TAZ'])

In [ ]:
taz_to_check

[]

In [ ]:
col=['indus',
'com_hotel',
'business',
'public',
'education',
'agri']

In [ ]:
taz['emp_check']=round(taz[col].sum(axis=1)-taz['emp_tot'])

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['emp_check']!=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check,emp_check


In [ ]:
taz_to_check=[]
for i in col:
    taz_to_check=taz_to_check+list(taz.loc[taz['{}'.format(i)]>0].loc[taz['emp_tot']<=0]['TAZ'])

In [ ]:
taz_to_check

[]

In [ ]:
taz.loc[taz['sector']!="Palestinian"].loc[taz['pop_emp_employed']>0].loc[taz['pop']<=0]

,TAZ,yosh,in_jerusalem_metropolin,jerusalem_city,sector,hh_total,pop,age0_4,age5_9,age10_14,age15_19,age20_24,age25_29,age30_34,age35_39,age40_44,age45_49,age50_54,age55_59,age60_64,age65_69,age70_74,age75up,emp_tot,indus,com_hotel,business,public,education,agri,student,univ,UO_Hi_Ed,pop_emp_employed,pop_check,emp_check
